# Maximal Static Expansion for efficient parallelization on GPU

## Context

### Polly
My GSoC project is part of Polly. Polly is a loop and data-locality optimizer for LLVM. The optimisations are made using a mathematical model called polyhedral model. It models the memory access of the program. After modeling, transformations (tilling, loop fusion, loop unrolling ...) can be applied on the model to improve data-locality and/or parallelization. The aim of my project was to implement a transformation called Maximal Static Expansion (MSE).

**TODO : Explain here dependences, memory access map, domain, range etc ...**

### Maximal static expansion
Data-dependences in a program can lead to a very bad automatic parallelization. Modern compilers use techniques to reduce the number of such dependences. One of them is Maximal Static Expansion. The MSE is a transformation which expand the memory access to and from Array or Scalar. The goal is to disambiguate memory accesses by assigning different memory locations to non-conflicting writes. This method is described in a paper written by Denis Barthou, Albert Cohen and Jean-Francois Collard.[^f1] 
Let take a example (from the article) to understand the principle :

In [4]:
int tmp;
for (int i = 0; i < N; i++) {
    tmp = i;
    for (int j = 0; j < N; j++) {
        tmp = tmp + i + j;
    }
    A[i] = tmp;
}

SyntaxError: invalid syntax (<ipython-input-4-cdf2a05d1563>, line 1)

The data-dependences induced by tmp make the two loops unparallelizable : the iteration j of the inner-loop needs value from the previous iteration and it is impossible to parrallelize the i-loop because tmp is used in all iterations.

If we expand the accesses to tmp according to the outermost loop, we can then parallelize the i-loop.

In [ ]:
int tmp_exp[N];
for (int i = 0; i < N; i++) {
    tmp_exp[i] = i;
    for (int j = 0; j < N; j++) {
        tmp_exp[i] = tmp_exp[i] + i + j;
    }
    A[i] = tmp_exp[i];
}

The accesses to tmp are now made to/from a different location for each iteration of the i-loop. It is then possible to execute the different iteration on different computation units (GPU, CPU ...).

### Static single assignement
Due to lack of time, I was not able to implement **maximal** static expansion but only fully-indexed expansion. The principle of fully-index expansion is that each write goes to a different memory location. 
Let see the idea on an example :

In [ ]:
int tmp;
for (int i = 0; i < N; i++) {
    tmp = i;
    for (int j = 0; j < N; j++) {
        B[j] = tmp + 3;
    }
    A[i] = B[i];
}

For the sake of simplicity, only the arrays will be expanded in this example. The fully expanded version is :

In [ ]:
int tmp;
for (int i = 0; i < N; i++) {
    tmp = i;
    for (int j = 0; j < N; j++) {
        B_exp[i][j] = tmp + 3;
    }
    A_exp[i] = B_exp[i][i];
}

The details of fully-indexed expansion will be discussed in the following sections.


## My work
My project is part of Polly. I am a french student but during the GSoC I was a student at the university of Passau, Germany. The LooPo team welcome me and more especially Andreas Simbürger, one of my GSoC mentor and my master thesis supervisor. My other GSoC mentor is Michael Kruse, one of the main contributor to Polly, actually working in France.
I'd like to thank all the people that help and guide me and more especially Andreas and Michael. 

### JSON bug fix
As first step in open source software development and to get familiar with Polly/LLVM development process, I fixed a open bug in Polly. Polly can import data from a JSON file (in case of Polly called jscop file). It can import new array, new memory access, new schedule or new context. In the previous implementation of JSONImporter, Polly did not check if the data in the jscop file were plausible and consistent before import. This can lead to failure in the remaining part of the Polly pipeline. My work was to implement plausibility and consistency checks. Details, diff and discussions can be found here : https://reviews.llvm.org/D32739. This patch has been merged into the actual version of Polly.

### Allocate array on heap
During transformation, Polly can create new arrays because it helps optimizing the program. Before GSoC, it was only possible to allocate array on the stack. It is sufficient for small arrays, but while doing expansion, we possibly handle very large arrays. For example, if we want to expand fully this simple code :


In [ ]:
for (int i = 0; i < N; i++)
  for (int j = 0; j < N; j++)
    for (int k = 0; k < N; k++)
      for (int l = 0; l < N; l++)
        A[l] = 3;

The expansion would lead to the following code :

In [ ]:
for (int i = 0; i < N; i++)
  for (int j = 0; j < N; j++)
    for (int k = 0; k < N; k++)
      for (int l = 0; l < N; l++)
        A_exp[i][j][k][l] = 3;

Depending on the value of N, A_exp can have a huge number of elements. If N = 100, we have $100*100*100*100 = 100000000 = 10^8$ elements ! Thus, the possibility to allocate array on heap was needed. 

The array allocation is implemented in the IslNodeBuilder. Here is the part of the code that do the allocation :

In [ ]:
void IslNodeBuilder::allocateNewArrays(BBPair StartExitBlocks) {
  for (auto &SAI : S.arrays()) {
    if (SAI->getBasePtr())
      continue;

    assert(SAI->getNumberOfDimensions() > 0 && SAI->getDimensionSize(0) &&
           "The size of the outermost dimension is used to declare newly "
           "created arrays that require memory allocation.");

    Type *NewArrayType = nullptr;

    // Get the size of the array = size(dim_1)*...*size(dim_n)
    uint64_t ArraySizeInt = 1;
    for (int i = SAI->getNumberOfDimensions() - 1; i >= 0; i--) {
      auto *DimSize = SAI->getDimensionSize(i);
      unsigned UnsignedDimSize = static_cast<const SCEVConstant *>(DimSize)
                                     ->getAPInt()
                                     .getLimitedValue();

      if (!NewArrayType)
        NewArrayType = SAI->getElementType();

      NewArrayType = ArrayType::get(NewArrayType, UnsignedDimSize);
      ArraySizeInt *= UnsignedDimSize;
    }

    if (SAI->isOnHeap()) {
      LLVMContext &Ctx = NewArrayType->getContext();

      // Get the IntPtrTy from the Datalayout
      auto IntPtrTy = DL.getIntPtrType(Ctx);

      // Get the size of the element type in bits
      unsigned Size = SAI->getElemSizeInBytes();

      // Insert the malloc call at polly.start
      auto InstIt = std::get<0>(StartExitBlocks)->getTerminator();
      auto *CreatedArray = CallInst::CreateMalloc(
          &*InstIt, IntPtrTy, SAI->getElementType(),
          ConstantInt::get(Type::getInt64Ty(Ctx), Size),
          ConstantInt::get(Type::getInt64Ty(Ctx), ArraySizeInt), nullptr,
          SAI->getName());

      SAI->setBasePtr(CreatedArray);

      // Insert the free call at polly.exiting
      CallInst::CreateFree(CreatedArray,
                           std::get<1>(StartExitBlocks)->getTerminator());

    } else {
      auto InstIt = Builder.GetInsertBlock()
                        ->getParent()
                        ->getEntryBlock()
                        .getTerminator();

      auto *CreatedArray = new AllocaInst(NewArrayType, DL.getAllocaAddrSpace(),
                                          SAI->getName(), &*InstIt);
      CreatedArray->setAlignment(PollyTargetFirstLevelCacheLineSize);
      SAI->setBasePtr(CreatedArray);
    }
  }
}

My work in this method is the size computation and the heap allocation part. The remaining code was already in place.

Let explain step by step the principle of the heap allocation.

First of all, to allocate array, we need to have the size of the memory chunk we want to allocate. To do that, we simply iterate over the dimension of the ScopArrayInfo and multiply the size of each dimensions. This is done by this code :

In [ ]:
    // Get the size of the array = size(dim_1)*...*size(dim_n)
    uint64_t ArraySizeInt = 1;
    for (int i = SAI->getNumberOfDimensions() - 1; i >= 0; i--) {
      auto *DimSize = SAI->getDimensionSize(i);
      unsigned UnsignedDimSize = static_cast<const SCEVConstant *>(DimSize)
                                     ->getAPInt()
                                     .getLimitedValue();

To actually do the expansion, we need to add a malloc call at the beginning of the polly section, called polly.start. After each malloc, a free must be present. The free call is added at polly.end. These two BasicBlock (polly.start and polly.end) are passed to allocateNewArray as a BBPair. We choose polly.start and polly.end as insertion points after a dense discussion with the polly community because this certify that there is no use-after-free (for instance in case of Scop in a loop) and that all memory cells allocated with a malloc are free'd when we don't need them anymore.

To get polly.start and polly.end, we modify executeScopConditionnaly such that it return both start block and end block of the scop. In the previous version of Polly, executeScopConditionnaly only return polly.start. People who wanted polly.end assume that there is no BasicBlock between polly.start and polly.end, so they just take the successor of polly.start. 

The malloc call insertion is made by this code :

In [ ]:
    auto *CreatedArray = CallInst::CreateMalloc(
              &*InstIt, IntPtrTy, SAI->getElementType(),
              ConstantInt::get(Type::getInt64Ty(Ctx), Size),
              ConstantInt::get(Type::getInt64Ty(Ctx), ArraySizeInt), nullptr,
              SAI->getName());

The free call insertion is made by this code :

In [ ]:
    CallInst::CreateFree(CreatedArray, std::get<1>(StartExitBlocks)->getTerminator());

Details, diff and discussions can be found here : https://reviews.llvm.org/D33688. This patch has been merged into the actual version of Polly.

### Array Fully indexed exp
#### Principle
As a first step in term of expansion, we choose to expand only arrays because we thought that it was an easy step to build expansion infrastructure. We also choose to no implement the **maximal** expansion in this step, to focus our efforts on the architecture of expansion : we implement a Fully-index expansion. The principle is that, for each array in the Scop, we expand the write to the array according to the loop nest and then we map the reads to the right iteration of the newly create ScopArrayInfo. Let see this on an example.



In [ ]:
int tmp;
for (int i = 0; i < N; i++) {
    tmp = i;
    for (int j = 0; j < N; j++) {
S:      B[j] = tmp + 3;
    }
T:  A[i] = B[i];
}

The write to B occurs inside the i and j loops. Therefore, the expanded version of B must be a two-dimensional array indexed by i and j. The write to A occurs inside the i loop only, therefore it would no need expansion. But for the sake of simplicity, we still create an expanded version of A. After write expansion, the code would look like that :

In [ ]:
int tmp;
for (int i = 0; i < N; i++) {
    tmp = i;
    for (int j = 0; j < N; j++) {
S:      B_exp[i][j] = tmp + 3;
    }
T:  A_exp[i] = B[i];
}

There is no read from A, so the expansion of A is done. There is a read from B in the statement T. At this step, we need the RAW dependences. In our case, statement T depends on statement S because the memory location reads by statement T is written by statetement S during j-loop iterations. The dependency looks like :

$$ \{ T[i] \rightarrow S[i][i] : 0\le i \le N \}$$

Now that we know that the statement T must read its value from the statement S at index [i,i], we only have to know the name of the expanded version of B and modify the read. After read expansion, the source code looks like that :

In [ ]:
int tmp;
for (int i = 0; i < N; i++) {
    tmp = i;
    for (int j = 0; j < N; j++) {
S:      B_exp[i][j] = tmp + 3;
    }
T:  A_exp[i] = B_exp[i][i];
}

More details can be found in these two articles[^f2][^f3].

#### Implementation
Let see now how this principle has been implemented in Polly.

Static expansion has been implemented as a ScopPass, which is a Pass triggered on every Scop detected by Polly. Guarded by an option, it is possible to ask Polly to do the expansion by adding **-polly-enable-mse** to clang or **-polly-mse** to opt command line. 

Here is the 'main' of static expansion. This code is straightforward and explains by itself the idea of expansion.

In [ ]:
  // Get the RAW Dependences.
  auto &DI = getAnalysis<DependenceInfo>();
  auto &D = DI.getDependences(Dependences::AL_Reference);
  auto Dependences = isl::give(D.getDependences(Dependences::TYPE_RAW));

  for (auto SAI : S.arrays()) {
    SmallPtrSet<MemoryAccess *, 4> AllWrites;
    SmallPtrSet<MemoryAccess *, 4> AllReads;
    if (!isExpandable(SAI, AllWrites, AllReads, S, Dependences))
      continue;

    auto TheWrite = *(AllWrites.begin());
    ScopArrayInfo *ExpandedArray = expandWrite(S, TheWrite);

    for (MemoryAccess *MA : AllReads)
      expandRead(S, MA, Dependences, ExpandedArray);
  }

The three first line is just the way to get dependences from Polly infrastructure. We request the DependenceInfo analysis for RAW dependences, using the Reference level statement. In a first iteration, we were using Statement Level statement. But this causes bugs. Full discussion and bug fixing can be found here :  https://reviews.llvm.org/D36791.

Then, we iterate over ScopArrayInfo in the Scop we are processing. We check if the ScopArrayInfo is expandable. If yes, we expand the write following the principle describe below and after we expand the reads.

TODO : expandability check
TODO : expand write
TODO : expand reads



Details, diff and discussions can be found here : https://reviews.llvm.org/D34982. This patch and the bug fixing one have been merged into the actual version of Polly.

clear deps : https://reviews.llvm.org/D36926
clears deps michael : https://reviews.llvm.org/D37010

### Scalar Fully indexed exp
Details, diff and discussions can be found here : https://reviews.llvm.org/D36647. This patch has been merged into the actual version of Polly.


## Evaluation

## Remaining work

### MAXIMAL expansion
### Select which SAI to expand

[^f1]: Denis Barthou, Albert Cohen, and Jean-François Collard. 2000. Maximal Static Expansion. Int. J. Parallel Program. 28, 3 (June 2000), 213-243. DOI=http://dx.doi.org/10.1023/A:1007500431910 
[^f2]: P. Feautrier. 1988. Array expansion. In Proceedings of the 2nd international conference on Supercomputing (ICS '88), J. Lenfant (Ed.). ACM, New York, NY, USA, 429-441. DOI=http://dx.doi.org/10.1145/55364.55406 
[^f3]: Dynamic Single Assignment. (n.d.). [ebook] Peter Vanbroekhoven. Available at: http://www.elis.ugent.be/aces/edegem2002/vanbroekhoven.pdf [Accessed 22 Aug. 2017].